# 1. Imports

In [ ]:
#pip install -U ydata-profiling
# !pip install ipywidgets

In [ ]:
from ydata_profiling import ProfileReport
#from matplotlib import pyplot as plt
#import seaborn as sns
import pandas as pd
import re
import json
import csv

# 1. Read in Data

In [6]:
file_path = "data/Asien/asien_tripfile.feather"

In [12]:
# Beispieldaten
#data = pd.read_csv(file_path, delimiter=';', quotechar='"', engine='python')

In [13]:
# Nicht für Beispieldaten, nur für csv
#data = pd.read_csv(file_path, delimiter=',', quotechar='"', engine='python', on_bad_lines='skip')

In [8]:
data_asien = pd.read_feather(file_path)

In [13]:
data_asien

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,header_line,entry_details
0,33902381,2024-05-01 09:14:42,AB,2359.0,1.0,BLR,service-acco,ASMMsgProcessor,"2024-05-01 09:14:42,420 INFO [3f326d134d9de45...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
1,33910427,2024-05-01 10:28:12,AB,2104.0,2.0,BLR,service-acco,ASMMsgProcessor,"2024-05-01 10:28:12,745 INFO [de24be75947007f...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
2,33910380,2024-05-01 10:28:11,AB,2109.0,2.0,BLR,service-acco,ASMMsgProcessor,"2024-05-01 10:28:11,301 INFO [df3a1cc7d574161...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
3,33909790,2024-05-01 10:28:12,AB,2110.0,2.0,BOM,service-acco,ASMMsgProcessor,"2024-05-01 10:28:12,189 INFO [9e66dfb1ffe9048...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
4,33902706,2024-05-01 09:14:41,AB,2127.0,2.0,BOM,service-acco,ASMMsgProcessor,"2024-05-01 09:14:41,884 INFO [829566f527b588f...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
...,...,...,...,...,...,...,...,...,...,...
710055,33875192,2024-05-01 04:07:01,AB,2372.0,30.0,GAU,service-acco,StorePaxDataAction,"2024-05-01 04:07:01,675 INFO [90c6642a105b86f...",TOTAL Pax: 131 Y: 131 Jump: 0 StandBy: ...
710056,33875261,2024-05-01 04:08:01,AB,2372.0,30.0,GAU,service-acco,StorePaxDataAction,"2024-05-01 04:08:01,279 INFO [76f51797d6d0b42...",com.onesystem.lc2.paxactuals.dto.PaxDataDTO [ ...
710057,33875264,2024-05-01 04:08:01,AB,2372.0,30.0,GAU,service-acco,StorePaxDataAction,"2024-05-01 04:08:01,281 INFO [76f51797d6d0b42...",TOTAL Pax: 131 Y: 131 Jump: 0 StandBy: ...
710058,33875306,2024-05-01 04:09:01,AB,2372.0,30.0,GAU,service-acco,StorePaxDataAction,"2024-05-01 04:09:01,465 INFO [ef79e34fa812d85...",com.onesystem.lc2.paxactuals.dto.PaxDataDTO [ ...


# 2. Understanding the Data

In [14]:
profile = ProfileReport(data_asien, title="Profiling Report")

In [ ]:
profile

In [ ]:
# @title creation_time vs count()

def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['creation_time']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'creation_time'}, axis=1)
              .sort_values('creation_time', ascending=True))
  xs = counted['creation_time']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = data.sort_values('creation_time', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('user_name')):
  _plot_series(series, series_name, i)
  fig.legend(title='user_name', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('creation_time')
_ = plt.ylabel('count()')

In [16]:
data_asien["action_name"].unique()

array(['ASMMsgProcessor', 'AcceptTransitDataAction',
       'AcceptTransitLoadAction', 'AcceptTransitNotocAction',
       'AssignLCCAction', 'AssignLoadplanAction',
       'AssignUnassignViewAction', 'CalculateWeightAndTrimAction',
       'CargoFinalAction', 'ChangeFlightLegStateAction',
       'CheckinMsgProcessor', 'CloseLegAction',
       'CreatePostDepartureMessagesAction',
       'CreateLoadingInstructionAction', 'CreateLoadsheetAction',
       'CreateZFWMessageAction', 'CreateBaggageLoadItemsAction',
       'ClearFlightsAction', 'InternalCreateLoadingInstructionAct',
       'InternalCreateLoadsheetAction', 'FlightPlanFiguresInMsgProcessor',
       'RampFinalAction', 'CrewMsgProcessor', 'PAXBOOKINGINMsgProcessor',
       'EstimateStorePaxDataAction', 'ReopenLegAction',
       'GetCabinConfigurationsAction', 'FuelDataInitializer',
       'SendPostDepartureMessagesAction',
       'SetActualBagWeightIndicatorAction', 'TransferCheckinDataAction',
       'StoreAircraftDataAction', 'Spe

# 3. Process the Data

In [ ]:
# List of all keys, including weights, indices, and specific aircraft parameters
weight_keys = [
    "START_WI weight",
    "START_WI index",
    "DO_WI weight",
    "DO_WI index",
    "PAX_WI weight",
    "PAX_WI index",
    "TOTAL_DEADLOAD_WI weight",
    "TOTAL_DEADLOAD_WI index",
    "TOTAL_LOAD_WI",
    "TOTAL_TRAFFIC_LOAD",
    "FUEL_INDEX",
    "AZFW",
    "ATOW",
    "ALAW",
    "FUEL_INDEX",
    "AFT_LIMIT_ZFW",
    "Start Weight",
    "Start Index",
    "Total Weight",
    "Index",
    "ATXW",
    "FWD_LIMIT_ZFW"
]

# Create the regex pattern dynamically from the list of keys
pattern = rf'({"|".join(map(re.escape, weight_keys))})\s*:\s*([0-9.]+)'

# Define a function to extract the relevant details
def extract_weight_details(data_string):
    extracted_data = re.findall(pattern, data_string)
    return {key.strip(): float(value) for key, value in extracted_data}

# Apply the function to the 'entry_details' column
data['extracted_data'] = data['entry_details'].astype(str).apply(extract_weight_details)

# Convert extracted dictionary into separate columns
for key in weight_keys:
    data[key] = data['extracted_data'].apply(lambda x: x.get(key, None))

In [ ]:
data.to_csv("_tripfile_extracted_weights.csv")